In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-203537
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-203537


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, please use it')
except ComputeTargetException:
    print('Creating a new cluster')
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)


Creating a new cluster
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,25,50,100,200,300,500,1000),
        '--max_iter': choice(50,100,200,250)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py',vm_size='STANDARD_D2_V2')

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    policy=policy,
                                    estimator=src,
                                    max_total_runs=16,
                                    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status()=="Completed")

RunId: HD_4986f363-fe92-4c46-a60c-da7ddb555c03
Web View: https://ml.azure.com/runs/HD_4986f363-fe92-4c46-a60c-da7ddb555c03?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-203537/workspaces/quick-starts-ws-203537&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-08-06T19:05:31.695602][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-08-06T19:05:32.3632856Z][SCHEDULER][INFO]Scheduling job, id='HD_4986f363-fe92-4c46-a60c-da7ddb555c03_0' 
[2022-08-06T19:05:32.5008775Z][SCHEDULER][INFO]Scheduling job, id='HD_4986f363-fe92-4c46-a60c-da7ddb555c03_1' 
[2022-08-06T19:05:32.6436545Z][SCHEDULER][INFO]Scheduling job, id='HD_4986f363-fe92-4c46-a60c-da7ddb555c03_2' 
[2022-08-06T19:05:32.701314][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-08-06T19:05:32.7572215Z][SCHEDULER][INFO]Scheduling job, id='HD_4986f363-fe92-4c46-a60c-da7ddb555c

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run=hyperdrive_run.get_best_run_by_primary_metric()
print("Best run Metrics:",best_run.get_metrics())
print("Best run details:",best_run.get_details())
print("best run file names:",best_run.get_file_names())

Best run Metrics: {'Regularization Strength:': 25.0, 'Max iterations:': 100, 'Accuracy': 0.9176024279210926}
Best run details: {'runId': 'HD_4986f363-fe92-4c46-a60c-da7ddb555c03_3', 'target': 'compute-cluster', 'status': 'Completed', 'startTimeUtc': '2022-08-06T19:09:37.132267Z', 'endTimeUtc': '2022-08-06T19:12:41.547805Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '4fcd897d-2cd9-427e-88c2-f1f726413fb9', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '25', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'compute-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'instanceTy

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = cpu_cluster,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [9]:
# Submit your automl run

automl_run = exp.submit(automl_config,show_output=False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5c1d61cd-f114-40c6-89e1-06d455502cad,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_5c1d61cd-f114-40c6-89e1-06d455502cad',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-08-06T19:18:52.823188Z',
 'endTimeUtc': '2022-08-06T20:04:14.032614Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-203537","workspace_name":"quick-starts-ws-203537","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_bat

In [10]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
print("Best run metrics:",best_run.get_metrics())
print("Best run details:", best_run.get_details())

Package:azureml-automl-runtime, training version:1.43.0, current version:1.42.0
Package:azureml-core, training version:1.43.0, current version:1.42.0
Package:azureml-dataset-runtime, training version:1.43.0, current version:1.42.0
Package:azureml-defaults, training version:1.43.0, current version:1.42.0
Package:azureml-interpret, training version:1.43.0, current version:1.42.0
Package:azureml-mlflow, training version:1.43.0.post1, current version:1.42.0
Package:azureml-pipeline-core, training version:1.43.0, current version:1.42.0
Package:azureml-responsibleai, training version:1.43.0, current version:1.42.0
Package:azureml-telemetry, training version:1.43.0, current version:1.42.0
Package:azureml-train-automl-client, training version:1.43.0, current version:1.42.0
Package:azureml-train-automl-runtime, training version:1.43.0.post1, current version:1.42.0
Package:azureml-train-core, training version:1.43.0, current version:1.42.0
Package:azureml-train-restclients-hyperdrive, training v

Run(Experiment: udacity-project,
Id: AutoML_5c1d61cd-f114-40c6-89e1-06d455502cad_34,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': '

In [11]:
#Deleting the cluster
cpu_cluster.delete()